In [3]:
from google.cloud import secretmanager
import pandas as pd
import numpy as np
from currensee.utils.db_utils import create_pg_engine
from sqlalchemy import text

import pandas as pd
import random
import os
import re
from faker import Faker


2025-07-11 01:24:17,166 - currensee.core.secrets - INFO - PROJECT_ID not found, attempting to load from .env file
2025-07-11 01:24:17,172 - currensee.core.secrets - INFO - Loaded environment from ./.env
2025-07-11 01:24:17,173 - currensee.core.secrets - INFO - SecretManager initialized with project_id: adsp-34002-on02-sopho-scribe


In [4]:
# Define DB_NAME
DB_NAME = "crm_outlook"

In [5]:
engine = create_pg_engine(db_name=DB_NAME)

In [6]:
# meetings between jane and her clients 
# have times be after 2023
df1 = pd.read_sql("SELECT * FROM meeting_data", con=engine)
df1.head()

,index,meeting_timestamp,host,host_email,invitees,invitee_emails,meeting_subject
0,0,2018-07-17 15:00:00,Jane Moneypenny,jane.moneypenny@bankwell.com,Cynthia Hobbs,cynthia.hobbs@abbvie.com,AbbVie - Introductory Call & Bankwell Services...
1,1,2018-07-24 14:00:00,Jane Moneypenny,jane.moneypenny@bankwell.com,Jennifer Phelps,jennifer.phelps@aerovironment.com,AeroVironment - Initial Discussion on Financia...
2,2,2018-08-07 10:00:00,Jane Moneypenny,jane.moneypenny@bankwell.com,Kyle Waters,kyle.waters@amedisys.com,Amedisys - Discussing Cash Management Needs
3,3,2018-08-15 11:00:00,Jane Moneypenny,jane.moneypenny@bankwell.com,Kyle Waters,kyle.waters@amedisys.com,Amedisys - Cash Concentration Services Demo
4,4,2018-08-29 14:00:00,Jane Moneypenny,jane.moneypenny@bankwell.com,Denise Moore,denise.moore@celestica.com,Celestica - Initial Discussion on Liquidity Ma...


In [7]:
# emails between jane and her clients
df2 = pd.read_sql("SELECT * FROM email_data", con=engine)
df2.head()

,index,email_timestamp,to_names,to_emails,from_name,from_email,email_subject,email_body
0,0,2018-07-15 09:30:00,Cynthia Hobbs,cynthia.hobbs@abbvie.com,Jane Moneypenny,jane.moneypenny@bankwell.com,Introduction - Bankwell Financial Services for...,"Hi Cynthia,\n\nMy name is Jane Moneypenny, and..."
1,1,2018-07-16 11:05:00,Jane Moneypenny,jane.moneypenny@bankwell.com,Cynthia Hobbs,cynthia.hobbs@abbvie.com,Re: Introduction - Bankwell Financial Services...,"Hi Jane,\n\nThank you for reaching out. AbbVie..."
2,2,2018-07-16 11:15:00,Cynthia Hobbs,cynthia.hobbs@abbvie.com,Jane Moneypenny,jane.moneypenny@bankwell.com,Re: Introduction - Bankwell Financial Services...,"Hi Cynthia,\n\nTuesday at 2 PM CT works perfec..."
3,3,2018-08-01 14:00:00,Jennifer Phelps,jennifer.phelps@aerovironment.com,Jane Moneypenny,jane.moneypenny@bankwell.com,Follow Up: AeroVironment Financial Needs,"Hi Jennifer,\n\nIt was a pleasure speaking wit..."
4,4,2018-08-02 10:15:00,Jane Moneypenny,jane.moneypenny@bankwell.com,Jennifer Phelps,jennifer.phelps@aerovironment.com,Re: Follow Up: AeroVironment Financial Needs,"Hi Jane,\n\nThanks for sending this over. The ..."


In [9]:
#INSERT INTO table_name (column1, column2, ...)
#VALUES (value1, value2, ...);

In [32]:
#View the while dataframe
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

***Code to create meetings more meetings***

In [16]:
import pandas as pd
from datetime import datetime, timedelta

# Meeting types
meeting_types = [
    "Regulatory Update",
    "Risk Management",
    "Annual Goal Alignment",
    "Tax Optimization",
    "ESG Investing",
    "Macro Update",
    "New Funds Offerings",
    "Benchmark Change"
]

# First meeting dates for clients
first_meetings = {
    "AbbVie": "2018-07-17 15:00:00",
    "AeroVironment": "2018-07-24 14:00:00",
    "Amedisys": "2018-08-07 10:00:00",
    "Celestica": "2018-08-15 11:00:00",
    "Compass": "2018-08-29 14:00:00",
    "GameStop Corp": "2018-09-25 10:00:00",
    "Guardant Health": "2018-10-11 16:00:00",
    "Hasbro": "2018-11-05 16:00:00",
    "Intuitive Surgical": "2018-11-14 11:00:00",
    "Ladder Capital Corp": "2018-12-12 13:00:00"
}

# Contacts: company -> (contact name, contact email)
contacts = {
    "AbbVie": ("Cynthia Hobbs", "cynthia.hobbs@abbvie.com"),
    "AeroVironment": ("Jennifer Phelps", "jennifer.phelps@aerovironment.com"),
    "Amedisys": ("Kyle Waters", "kyle.waters@amedisys.com"),
    "Celestica": ("Denise Moore", "denise.moore@celestica.com"),
    "Compass": ("Adam Clay", "adam.clay@compass.com"),
    "GameStop Corp": ("Amy Winters", "amy.winters@gamestopcorp.com"),
    "Guardant Health": ("Roberto Martin", "roberto.martin@guardanthealth.com"),
    "Hasbro": ("Jessica Palmer", "jessica.palmer@hasbro.com"),
    "Intuitive Surgical": ("Michelle Jenkins", "michelle.jenkins@intuitivesurgical.com"),
    "Ladder Capital Corp": ("Ronnie Gray", "ronnie.gray@laddercapitalcorp.com"),
}

# Blocked timestamps (±1 hour around actual meetings)
blocked_times = [
    '2018-07-17 15:00:00', '2018-07-24 14:00:00', '2018-08-07 10:00:00', '2018-08-15 11:00:00',
    '2018-08-29 14:00:00', '2018-09-25 10:00:00', '2018-10-11 16:00:00', '2018-11-05 16:00:00',
    '2018-11-14 11:00:00', '2018-12-12 13:00:00', '2019-01-30 14:00:00', '2019-02-26 09:00:00',
    '2019-03-05 13:00:00', '2019-03-20 14:30:00', '2019-04-10 14:00:00', '2019-04-24 11:30:00',
    '2019-05-17 16:00:00', '2019-07-30 14:00:00', '2019-09-17 10:00:00', '2019-10-21 14:00:00',
    '2019-10-23 14:00:00', '2019-10-24 11:00:00', '2020-01-28 14:00:00', '2020-01-29 11:00:00',
    '2020-03-09 10:00:00', '2020-04-07 10:00:00', '2020-04-21 14:00:00', '2020-04-30 16:00:00',
    '2020-04-27 14:00:00', '2020-07-07 12:00:00', '2020-07-22 16:00:00', '2020-07-29 14:00:00',
    '2020-07-30 11:00:00', '2020-09-08 15:00:00', '2020-09-15 14:00:00', '2020-10-29 14:00:00',
    '2020-10-30 11:00:00', '2021-01-26 14:00:00', '2021-01-27 11:00:00', '2021-02-15 14:00:00',
    '2021-04-06 13:00:00', '2021-04-06 09:00:00', '2021-04-28 14:00:00', '2021-05-11 14:30:00',
    '2021-06-15 14:00:00', '2021-06-29 10:00:00', '2021-07-28 14:00:00', '2021-07-29 14:00:00',
    '2021-07-30 11:00:00', '2021-08-25 10:00:00', '2021-09-21 14:00:00', '2021-10-01 11:00:00',
    '2021-10-27 14:00:00', '2021-10-28 11:00:00', '2021-10-29 10:00:00', '2021-12-07 17:00:00',
    '2022-01-25 14:00:00', '2022-01-26 11:00:00', '2022-04-04 11:00:00', '2022-04-06 09:00:00',
    '2022-04-27 14:00:00', '2022-04-28 11:00:00', '2022-06-15 11:00:00', '2022-07-26 14:00:00',
    '2022-07-27 11:00:00', '2022-10-25 14:00:00', '2022-10-26 11:00:00', '2023-01-10 10:00:00',
    '2023-01-24 14:00:00', '2023-01-25 11:00:00', '2023-01-26 14:00:00', '2023-04-25 14:00:00',
    '2023-04-26 11:00:00', '2023-06-06 11:00:00', '2023-07-25 14:00:00', '2023-07-26 14:00:00',
    '2023-08-01 14:00:00', '2023-08-23 10:00:00', '2023-10-24 14:00:00', '2023-10-25 11:00:00',
    '2023-12-05 16:00:00', '2023-12-11 11:00:00', '2024-01-23 14:00:00', '2024-01-24 11:00:00',
    '2024-01-25 14:00:00', '2024-01-29 11:00:00', '2024-02-06 15:00:00', '2024-02-13 14:00:00',
    '2024-02-20 11:00:00', '2024-02-27 10:00:00', '2024-03-05 13:00:00', '2024-03-12 09:00:00',
    '2024-03-19 16:00:00', '2024-03-26 11:00:00', '2024-02-04 14:30:00'
]


# Create blocked intervals ±1 hour
blocked_intervals = []
for ts in blocked_times:
    dt = datetime.strptime(ts, "%Y-%m-%d %H:%M:%S")
    blocked_intervals.append((dt - timedelta(hours=1), dt + timedelta(hours=1)))

def is_valid_time(candidate):
    return all(not (start <= candidate <= end) for start, end in blocked_intervals)

# Generate meetings DataFrame
rows = []
for client, first_time_str in first_meetings.items():
    first_time = datetime.strptime(first_time_str, "%Y-%m-%d %H:%M:%S")
    count = 0
    step = 0
    while count < len(meeting_types):
        candidate_time = first_time + timedelta(days=14 * (step + 1), hours=1)
        if is_valid_time(candidate_time):
            contact_name, contact_email = contacts.get(client, ("Unknown", "unknown@example.com"))
            rows.append({
                "meeting_timestamp": candidate_time,
                "host": "Jane Moneypenny",
                "host_email": "jane.moneypenny@bankwell.com",
                "invitees": client,  # Company name only
                "invitee_emails": contact_email,  # Actual contact email
                "meeting_subject": f"{client} - {meeting_types[count]}"
            })
            count += 1
        step += 1  # Try next candidate time

df_meetings = pd.DataFrame(rows)


In [17]:
# Original DataFrame for Jane
df_meetings_jane = pd.DataFrame(rows)

# Create Ryan's copy with updated host details
df_meetings_ryan = df_meetings_jane.copy()
df_meetings_ryan["host"] = "Ryan Holloway"
df_meetings_ryan["host_email"] = "ryan.holloway@bankwell.com"

# Combine both into one DataFrame
df_meetings_combined = pd.concat([df_meetings_jane, df_meetings_ryan], ignore_index=True)

# Optional: sort by timestamp or any other field
df_meetings_combined = df_meetings_combined.sort_values(by="meeting_timestamp").reset_index(drop=True)

In [26]:
df_meetings_combined

,meeting_timestamp,host,host_email,invitees,invitee_emails,meeting_subject
0,2018-07...,Jane M...,jane.m...,AbbVie,cynthi...,AbbVie...
1,2018-07...,Ryan H...,ryan.h...,AbbVie,cynthi...,AbbVie...
2,2018-08...,Jane M...,jane.m...,AeroVi...,jennif...,AeroVi...
3,2018-08...,Ryan H...,ryan.h...,AeroVi...,jennif...,AeroVi...
4,2018-08...,Jane M...,jane.m...,AbbVie,cynthi...,AbbVie...
...,...,...,...,...,...,...
155,2019-03...,Ryan H...,ryan.h...,Intuit...,michel...,Intuit...
156,2019-04...,Ryan H...,ryan.h...,Ladder...,ronnie...,Ladder...
157,2019-04...,Jane M...,jane.m...,Ladder...,ronnie...,Ladder...
158,2019-04...,Jane M...,jane.m...,Ladder...,ronnie...,Ladder...


***Code to create meetings more emails***

In [21]:

import pandas as pd
from datetime import datetime, timedelta
from faker import Faker
import random

fake = Faker()

# Meeting types and first meetings
meeting_types = [
    "Regulatory Update",
    "Risk Management",
    "Annual Goal Alignment",
    "Tax Optimization",
    "ESG Investing",
    "Macro Update",
    "New Funds Offerings",
    "Benchmark Change"
]

first_meetings = {
    "AbbVie": "2018-07-17 15:00:00",
    "AeroVironment": "2018-07-24 14:00:00",
    "Amedisys": "2018-08-07 10:00:00",
    "Celestica": "2018-08-15 11:00:00",
    "Compass": "2018-08-29 14:00:00",
    "GameStop Corp": "2018-09-25 10:00:00",
    "Guardant Health": "2018-10-11 16:00:00",
    "Hasbro": "2018-11-05 16:00:00",
    "Intuitive Surgical": "2018-11-14 11:00:00",
    "Ladder Capital Corp": "2018-12-12 13:00:00"
}

blocked_times_str = [
    '2018-07-17 15:00:00', '2018-07-24 14:00:00', '2018-08-07 10:00:00', '2018-08-15 11:00:00',
    '2018-08-29 14:00:00', '2018-09-25 10:00:00', '2018-10-11 16:00:00', '2018-11-05 16:00:00',
    '2018-11-14 11:00:00', '2018-12-12 13:00:00', '2019-01-30 14:00:00', '2019-02-26 09:00:00',
    '2019-03-05 13:00:00', '2019-03-20 14:30:00', '2019-04-10 14:00:00', '2019-04-24 11:30:00',
    '2019-05-17 16:00:00', '2019-07-30 14:00:00', '2019-09-17 10:00:00', '2019-10-21 14:00:00',
    '2019-10-23 14:00:00', '2019-10-24 11:00:00', '2020-01-28 14:00:00', '2020-01-29 11:00:00',
    '2020-03-09 10:00:00', '2020-04-07 10:00:00', '2020-04-21 14:00:00', '2020-04-30 16:00:00',
    '2020-04-27 14:00:00', '2020-07-07 12:00:00', '2020-07-22 16:00:00', '2020-07-29 14:00:00',
    '2020-07-30 11:00:00', '2020-09-08 15:00:00', '2020-09-15 14:00:00', '2020-10-29 14:00:00',
    '2020-10-30 11:00:00', '2021-01-26 14:00:00', '2021-01-27 11:00:00', '2021-02-15 14:00:00',
    '2021-04-06 13:00:00', '2021-04-06 09:00:00', '2021-04-28 14:00:00', '2021-05-11 14:30:00',
    '2021-06-15 14:00:00', '2021-06-29 10:00:00', '2021-07-28 14:00:00', '2021-07-29 14:00:00',
    '2021-07-30 11:00:00', '2021-08-25 10:00:00', '2021-09-21 14:00:00', '2021-10-01 11:00:00',
    '2021-10-27 14:00:00', '2021-10-28 11:00:00', '2021-10-29 10:00:00', '2021-12-07 17:00:00',
    '2022-01-25 14:00:00', '2022-01-26 11:00:00', '2022-04-04 11:00:00', '2022-04-06 09:00:00',
    '2022-04-27 14:00:00', '2022-04-28 11:00:00', '2022-06-15 11:00:00', '2022-07-26 14:00:00',
    '2022-07-27 11:00:00', '2022-10-25 14:00:00', '2022-10-26 11:00:00', '2023-01-10 10:00:00',
    '2023-01-24 14:00:00', '2023-01-25 11:00:00', '2023-01-26 14:00:00', '2023-04-25 14:00:00',
    '2023-04-26 11:00:00', '2023-06-06 11:00:00', '2023-07-25 14:00:00', '2023-07-26 14:00:00',
    '2023-08-01 14:00:00', '2023-08-23 10:00:00', '2023-10-24 14:00:00', '2023-10-25 11:00:00',
    '2023-12-05 16:00:00', '2023-12-11 11:00:00', '2024-01-23 14:00:00', '2024-01-24 11:00:00',
    '2024-01-25 14:00:00', '2024-01-29 11:00:00', '2024-02-06 15:00:00', '2024-02-13 14:00:00',
    '2024-02-20 11:00:00', '2024-02-27 10:00:00', '2024-03-05 13:00:00', '2024-03-12 09:00:00',
    '2024-03-19 16:00:00', '2024-03-26 11:00:00', '2024-02-04 14:30:00'
]



blocked_times = set(datetime.strptime(ts, "%Y-%m-%d %H:%M:%S") for ts in blocked_times_str)

def is_blocked(ts):
    return ts in blocked_times

def random_email_time(before_time, min_hours_before=2, max_hours_before=48):
    for _ in range(100):
        delta_hours = random.randint(min_hours_before, max_hours_before)
        candidate = before_time - timedelta(hours=delta_hours)
        candidate = candidate.replace(minute=random.choice([0,15,30,45]), second=0, microsecond=0)
        if not is_blocked(candidate):
            return candidate
    raise ValueError("Cannot find valid email timestamp")

# Generate client contacts
client_contacts = {}
for client in first_meetings.keys():
    name = fake.name()
    clean_name = name.lower().replace(" ", ".").replace("'", "")
    clean_client = client.lower().replace(" ", "").replace(".", "")
    email = f"{clean_name}@{clean_client}.com"
    client_contacts[client] = (name, email)

# Meeting data generation
rows_meetings = []
for client, first_time_str in first_meetings.items():
    first_time = datetime.strptime(first_time_str, "%Y-%m-%d %H:%M:%S")
    for i, meeting_type in enumerate(meeting_types):
        meeting_time = first_time + timedelta(days=14 * (i + 1))
        rows_meetings.append({
            "meeting_timestamp": meeting_time,
            "host": "Jane Moneypenny",
            "host_email": "jane.moneypenny@bankwell.com",
            "invitees": f"{client} Treasury Team",
            "invitee_emails": f"treasury@{client.lower().replace(' ', '').replace('.', '')}.com",
            "meeting_subject": f"{client} - {meeting_type}"
        })

df_meetings = pd.DataFrame(rows_meetings)

# Varied email templates for Jane
jane_email_templates = {
    "Regulatory Update": [
        "Dear {client_name},\n\nI wanted to remind you of our upcoming regulatory update meeting. Let me know if there are any specific regulations you'd like to discuss.\n\nBest regards,\nJane",
        "Hi {client_name},\n\nLooking forward to catching up on regulatory changes during our next meeting. Feel free to send any questions in advance.\n\nCheers,\nJane",
        "Hello {client_name},\n\nOur next meeting will cover important regulatory updates. Please share any topics you want me to prepare.\n\nRegards,\nJane",
        "Dear {client_name},\n\nPlease find attached a briefing note for our upcoming regulatory discussion. I’d appreciate any feedback on focus areas you care about.\n\nBest,\nJane",
        "There are a few SEC-related updates I think will be relevant to our next conversation. Let me know if you want deeper analysis, {client_name}.\n\nCheers,\nJane",
        "Good morning {client_name},\n\nI’ll be covering some cross-border compliance updates. Do you need a briefing on any non-U.S. jurisdictions?\n\nWarm regards,\nJane"
    ],
    "Risk Management": [
        "Dear {client_name},\n\nAhead of our risk management session, please think about any concerns or risks you'd like to address.\n\nBest,\nJane",
        "Hi {client_name},\n\nOur upcoming meeting focuses on risk management. Let me know if there are specific risks you want to explore.\n\nThanks,\nJane",
        "Hello {client_name},\n\nJust touching base before our risk management discussion. Please share any risk areas you'd like to highlight.\n\nWarmly,\nJane",
        "Looking forward to our discussion on current risk exposures, {client_name}. Please let me know if there are any specific areas you'd like me to prepare for.\n\nBest,\nJane",
        "Hi {client_name},\n\nLet’s ensure our risk review is comprehensive—send over any recent developments or exposures we should cover.\n\nThanks,\nJane",
        "We’ll be reviewing strategic risk metrics in our next meeting. Should I include simulations or historical stress events, {client_name}?\n\nBest,\nJane"
    ],
    "Annual Goal Alignment": [
        "Dear {client_name},\n\nAs we prepare for our annual planning meeting, I’d love your input on strategic priorities.\n\nBest regards,\nJane",
        "Hi {client_name},\n\nLooking forward to aligning on your 12-month goals. Let me know if you’d like me to prepare anything specific.\n\nThanks,\nJane",
        "Hello {client_name},\n\nAhead of our annual goal alignment session, feel free to send along your draft targets or any focus areas.\n\nRegards,\nJane",
        "Hi {client_name},\n\nLet’s come prepared with both tactical and long-term objectives—we’ll chart milestones together.\n\nCheers,\nJane",
        "Good day {client_name},\n\nThinking through how we can measure progress against your top 3 goals—any insights before our call?\n\nSincerely,\nJane"
    ],
    "Tax Optimization": [
        "Dear {client_name},\n\nI’ll walk you through some of the latest opportunities for tax deferral and loss harvesting. Let me know if there’s anything urgent.\n\nRegards,\nJane",
        "Hi {client_name},\n\nLooking forward to exploring how we can reduce your tax burden this quarter. Please bring any recent tax docs if applicable.\n\nThanks,\nJane",
        "Hello {client_name},\n\nLet’s dive into structured solutions and timing strategies in our upcoming session.\n\nBest,\nJane",
        "There’s been an IRS guidance change that may open a new planning path, {client_name}. I’ll explain during our meeting.\n\nCheers,\nJane",
        "If there’s any capital gains activity this year, let’s factor that into our discussion, {client_name}.\n\nWarm regards,\nJane"
    ],
    "ESG Investing": [
        "Hi {client_name},\n\nI’ve gathered a few ESG-aligned funds that might suit your mandates. I’ll walk you through the performance vs. benchmarks.\n\nBest,\nJane",
        "Dear {client_name},\n\nLet’s review ESG screening criteria in our meeting. Any new sustainability goals from your team I should know about?\n\nThanks,\nJane",
        "Hello {client_name},\n\nI’ve been exploring green bond options—happy to show you what’s available this cycle.\n\nRegards,\nJane",
        "Hi {client_name},\n\nWe’ll touch on ESG ratings and exclusions as part of our portfolio review.\n\nWarmly,\nJane",
        "Good day {client_name},\n\nI’ve updated our ESG scoring dashboard and would love your feedback.\n\nSincerely,\nJane"
    ],
    "Macro Update": [
        "Hi {client_name},\n\nI’ve pulled together key macroeconomic data and forecasts. Let’s walk through what may affect your portfolio.\n\nCheers,\nJane",
        "Dear {client_name},\n\nAhead of our macro update, I’m including charts on inflation, rates, and global growth. Let me know if you'd like anything added.\n\nThanks,\nJane",
        "Hello {client_name},\n\nPlenty to discuss this cycle—Fed outlook, global demand, and capital flows. I’ll give you a synthesis.\n\nWarm regards,\nJane",
        "Expect updates on policy shifts and market trends in our next discussion, {client_name}.\n\nBest,\nJane",
        "I’ll frame the macro view from a risk and opportunity lens. Do you want sector-level analysis too, {client_name}?\n\nSincerely,\nJane"
    ],
    "New Funds Offerings": [
        "Hi {client_name},\n\nWe’ve launched two new funds—one growth-oriented and one fixed income. I’ll share details during the call.\n\nBest,\nJane",
        "You might be interested in a new fund focused on sustainable yield, {client_name}. Let me know if you’d like materials in advance.\n\nThanks,\nJane",
        "There’s a new thematic fund I’d like to flag—it might align with your current allocation, {client_name}.\n\nWarm regards,\nJane",
        "Hi {client_name},\n\nAhead of the meeting, take a glance at the fund summary I attached. Excited to get your thoughts.\n\nCheers,\nJane",
        "We’ll cover the new offerings next week. Bring any questions or mandates you're exploring, {client_name}.\n\nKindly,\nJane"
    ],
    "Benchmark Change": [
        "Hi {client_name},\n\nThere’s been a proposed change to your benchmark index. Let’s review the rationale and impact during our call.\n\nBest,\nJane",
        "I’ll show the side-by-side comparison of old vs. new benchmarks so we can validate fit, {client_name}.\n\nThanks,\nJane",
        "Benchmark re-alignment can affect your performance metrics—I’ll walk you through it, {client_name}.\n\nWarm regards,\nJane",
        "We’re proposing a shift to a blended index. I’ll share supporting analysis in our meeting, {client_name}.\n\nCheers,\nJane",
        "We’ll cover historical tracking error and risk-return fit with the new benchmark, {client_name}.\n\nSincerely,\nJane"
    ]
}


# Varied client questions templates keyed by meeting type
client_email_templates = {
    "Regulatory Update": [
        "Hi Jane,\n\nThanks for setting this up. Can we include any upcoming FINRA changes in the discussion?\n\nBest,\n{client_name}",
        "Dear Jane,\n\nLooking forward to the regulatory session. Could you touch on any state-level compliance updates too?\n\nRegards,\n{client_name}",
        "Hello Jane,\n\nWould it be possible to review the new filing timelines as part of our meeting?\n\nThanks,\n{client_name}",
        "Jane,\n\nCould we also discuss the latest anti-money laundering requirements?\n\nAppreciate it,\n{client_name}",
        "Hi Jane,\n\nQuick question before our meeting—any recent shifts in cross-border disclosure rules we should note?\n\nBest,\n{client_name}"
    ],
    "Risk Management": [
        "Hi Jane,\n\nWould love to explore potential FX exposure in our upcoming session.\n\nThanks,\n{client_name}",
        "Dear Jane,\n\nCould you prepare something on our counterparty risk position as well?\n\nRegards,\n{client_name}",
        "Hello Jane,\n\nJust checking if we’ll cover risk limits and tolerance thresholds during the session.\n\nThanks,\n{client_name}",
        "Jane,\n\nCan you walk me through our current interest rate hedges again? Want to confirm alignment.\n\nBest,\n{client_name}",
        "Hi Jane,\n\nAny updates on market risk scenarios that we should factor into our dashboard?\n\nCheers,\n{client_name}"
    ],
    "Annual Goal Alignment": [
        "Hi Jane,\n\nFor our planning session, I’d like to explore quarterly targets aligned with our liquidity goals.\n\nBest,\n{client_name}",
        "Dear Jane,\n\nCan we discuss metrics we should track against our new debt structure?\n\nThanks,\n{client_name}",
        "Hello Jane,\n\nWould you be able to show projections tied to last year’s goal outcomes?\n\nWarmly,\n{client_name}",
        "Jane,\n\nCould we bring in someone from your team who focuses on long-term capital planning?\n\nThanks,\n{client_name}",
        "Hi Jane,\n\nI’d love to hear your view on aligning our treasury KPIs with broader finance objectives.\n\nRegards,\n{client_name}"
    ],
    "Tax Optimization": [
        "Hi Jane,\n\nWould appreciate a review of potential deferral options for our year-end bonuses.\n\nThanks,\n{client_name}",
        "Dear Jane,\n\nCould we factor in the new state tax rules when optimizing our setup?\n\nWarm regards,\n{client_name}",
        "Jane,\n\nDo you have any modeling on short-term vs. long-term gains treatments?\n\nThanks,\n{client_name}",
        "Hello Jane,\n\nOne quick request—let’s include charitable deductions in the analysis.\n\nBest,\n{client_name}",
        "Hi Jane,\n\nCan you include tax drag impacts under current allocation?\n\nThanks,\n{client_name}"
    ],
    "ESG Investing": [
        "Hi Jane,\n\nWill we look at ESG fund flows or just performance vs. traditional indices?\n\nThanks,\n{client_name}",
        "Jane,\n\nCould you break down ESG screening criteria again, especially for energy holdings?\n\nRegards,\n{client_name}",
        "Dear Jane,\n\nAny updates on impact-focused funds you’d recommend for our review?\n\nBest,\n{client_name}",
        "Hello Jane,\n\nCan we also compare carbon scores across the proposed portfolios?\n\nCheers,\n{client_name}",
        "Hi Jane,\n\nDo you have a view on integrating ESG in short-term instruments?\n\nThanks,\n{client_name}"
    ],
    "Macro Update": [
        "Jane,\n\nCan we allocate a bit more time to global inflation forecasts?\n\nBest,\n{client_name}",
        "Hi Jane,\n\nWould appreciate your take on central bank policy divergence.\n\nThanks,\n{client_name}",
        "Dear Jane,\n\nCould you include a chart on commodity trends this cycle?\n\nRegards,\n{client_name}",
        "Hello Jane,\n\nJust flagging—geopolitical risks may merit some attention this meeting.\n\nThanks,\n{client_name}",
        "Hi Jane,\n\nAny chance we can look at cross-asset volatility in your macro outlook?\n\nCheers,\n{client_name}"
    ],
    "New Funds Offerings": [
        "Hi Jane,\n\nCould you send over fund sheets in advance for review?\n\nThanks,\n{client_name}",
        "Jane,\n\nDo any of the new funds support daily liquidity? That’s key for us.\n\nBest,\n{client_name}",
        "Dear Jane,\n\nWould love to hear if there’s a low-volatility strategy among the new products.\n\nRegards,\n{client_name}",
        "Hello Jane,\n\nI’m curious if any of the offerings lean into private credit or alternatives.\n\nThanks,\n{client_name}",
        "Hi Jane,\n\nAre there ESG-screened versions of these funds as well?\n\nWarm regards,\n{client_name}"
    ],
    "Benchmark Change": [
        "Jane,\n\nLooking forward to this one—can we dig into how the new benchmark affects our Sharpe ratio?\n\nThanks,\n{client_name}",
        "Hi Jane,\n\nAny visuals you could share to show tracking differences?\n\nBest,\n{client_name}",
        "Dear Jane,\n\nCan you walk me through sector weight shifts with the new benchmark?\n\nThanks,\n{client_name}",
        "Hello Jane,\n\nI’m curious about volatility vs. the previous benchmark. Can we include that?\n\nCheers,\n{client_name}",
        "Jane,\n\nJust flagging I’ll need to report this internally, so if there’s a summary memo, that’d be great.\n\nBest,\n{client_name}"
    ]
}

# For meeting types without templates, fallback to a generic one
def get_random_template(templates_dict, key, client_name):
    templates = templates_dict.get(key)
    if not templates:
        # Generic fallback
        return f"Dear {client_name},\n\nLooking forward to our discussion on {key}. Let me know if you want to cover anything specific.\n\nBest,\nJane"
    else:
        template = random.choice(templates)
        return template.format(client_name=client_name)

def get_random_question(questions_dict, key):
    questions = questions_dict.get(key)
    if not questions:
        return "I have some questions regarding the meeting agenda."
    else:
        return random.choice(questions)

email_rows = []
idx = 0

for _, meeting in df_meetings_combined.iterrows():
    meeting_time = meeting['meeting_timestamp']
    client = meeting['invitees'].replace(' Treasury Team', '')
    meeting_type = meeting['meeting_subject'].split(' - ')[1]
    host_name = meeting['host']
    host_email = meeting['host_email']

    # Fixed contact lookup
    if client in contact_lookup:
        client_name = contact_lookup[client]['invitees']
        client_email = contact_lookup[client]['invitee_emails']
    else:
        client_name = f"{client} Treasury Team"
        client_email = meeting['invitee_emails']

    # --- Host (Jane or Ryan) sends email, but body signed by Jane ---
    email_time = random_email_time(meeting_time, 2, 48)
    subject = f"{client} - {meeting_type}"
    email_body = get_random_template(jane_email_templates, meeting_type, client_name)
    # ⚠️ No replacement of "Dear Jane" here

    email_rows.append({
        "index": idx,
        "email_timestamp": email_time,
        "from_name": host_name,
        "from_email": host_email,
        "to_name": client_name,
        "to_email": client_email,
        "email_subject": subject,
        "email_body": email_body,
        "meeting_timestamp": meeting_time,
        "meeting_subject": meeting['meeting_subject']
    })
    idx += 1

    # --- Client replies to host, but says "Dear Jane" always ---
    min_reply_time = email_time + timedelta(hours=1)
    max_reply_time = meeting_time - timedelta(hours=1)

    for _ in range(100):
        if min_reply_time >= max_reply_time:
            client_email_time = min_reply_time
            break
        delta_hours = random.randint(1, max(1, int((max_reply_time - min_reply_time).total_seconds() // 3600)))
        client_email_time = min_reply_time + timedelta(hours=delta_hours)
        client_email_time = client_email_time.replace(minute=random.choice([0,15,30,45]), second=0, microsecond=0)
        if not is_blocked(client_email_time) and client_email_time < meeting_time:
            break
    else:
        client_email_time = min_reply_time

    client_subject = f"Re: {client} - {meeting_type}"
    client_question = get_random_question(client_email_templates, meeting_type)
    client_body = (
        f"Dear Jane,\n\n"
        f"Thank you for your email. {client_question}\n\n"
        f"Best regards,\n{client_name}"
    )

    email_rows.append({
        "index": idx,
        "email_timestamp": client_email_time,
        "from_name": client_name,
        "from_email": client_email,
        "to_name": host_name,
        "to_email": host_email,
        "email_subject": client_subject,
        "email_body": client_body,
        "meeting_timestamp": meeting_time,
        "meeting_subject": meeting['meeting_subject']
    })
    idx += 1

df_emails = pd.DataFrame(email_rows)
df_emails = df_emails.sort_values("email_timestamp").reset_index(drop=True)



In [23]:
#ADJUSTMENT TO EMAILS DF

df_emails_new  = df_emails.rename(columns={
    "to_email": "to_emails",
    "to_name": "to_names"
}).drop(columns=["meeting_timestamp", "meeting_subject"])

# Ensure pandas shows full column width and all rows
pd.set_option('display.max_colwidth', 10)
pd.set_option('display.max_rows', 10)


In [ ]:

# import pandas as pd
# from datetime import datetime, timedelta
# from faker import Faker
# import random

# fake = Faker()

# # Meeting types and first meetings
# meeting_types = [
#     "Regulatory Update",
#     "Risk Management",
#     "Annual Goal Alignment",
#     "Tax Optimization",
#     "ESG Investing",
#     "Macro Update",
#     "New Funds Offerings",
#     "Benchmark Change"
# ]

# first_meetings = {
#     "AbbVie": "2018-07-17 15:00:00",
#     "AeroVironment": "2018-07-24 14:00:00",
#     "Amedisys": "2018-08-07 10:00:00",
#     "Celestica": "2018-08-15 11:00:00",
#     "Compass": "2018-08-29 14:00:00",
#     "GameStop Corp": "2018-09-25 10:00:00",
#     "Guardant Health": "2018-10-11 16:00:00",
#     "Hasbro": "2018-11-05 16:00:00",
#     "Intuitive Surgical": "2018-11-14 11:00:00",
#     "Ladder Capital Corp": "2018-12-12 13:00:00"
# }

# blocked_times_str = [
#     '2018-07-17 15:00:00', '2018-07-24 14:00:00', '2018-08-07 10:00:00', '2018-08-15 11:00:00',
#     '2018-08-29 14:00:00', '2018-09-25 10:00:00', '2018-10-11 16:00:00', '2018-11-05 16:00:00',
#     '2018-11-14 11:00:00', '2018-12-12 13:00:00', '2019-01-30 14:00:00', '2019-02-26 09:00:00',
#     '2019-03-05 13:00:00', '2019-03-20 14:30:00', '2019-04-10 14:00:00', '2019-04-24 11:30:00',
#     '2019-05-17 16:00:00', '2019-07-30 14:00:00', '2019-09-17 10:00:00', '2019-10-21 14:00:00',
#     '2019-10-23 14:00:00', '2019-10-24 11:00:00', '2020-01-28 14:00:00', '2020-01-29 11:00:00',
#     '2020-03-09 10:00:00', '2020-04-07 10:00:00', '2020-04-21 14:00:00', '2020-04-30 16:00:00',
#     '2020-04-27 14:00:00', '2020-07-07 12:00:00', '2020-07-22 16:00:00', '2020-07-29 14:00:00',
#     '2020-07-30 11:00:00', '2020-09-08 15:00:00', '2020-09-15 14:00:00', '2020-10-29 14:00:00',
#     '2020-10-30 11:00:00', '2021-01-26 14:00:00', '2021-01-27 11:00:00', '2021-02-15 14:00:00',
#     '2021-04-06 13:00:00', '2021-04-06 09:00:00', '2021-04-28 14:00:00', '2021-05-11 14:30:00',
#     '2021-06-15 14:00:00', '2021-06-29 10:00:00', '2021-07-28 14:00:00', '2021-07-29 14:00:00',
#     '2021-07-30 11:00:00', '2021-08-25 10:00:00', '2021-09-21 14:00:00', '2021-10-01 11:00:00',
#     '2021-10-27 14:00:00', '2021-10-28 11:00:00', '2021-10-29 10:00:00', '2021-12-07 17:00:00',
#     '2022-01-25 14:00:00', '2022-01-26 11:00:00', '2022-04-04 11:00:00', '2022-04-06 09:00:00',
#     '2022-04-27 14:00:00', '2022-04-28 11:00:00', '2022-06-15 11:00:00', '2022-07-26 14:00:00',
#     '2022-07-27 11:00:00', '2022-10-25 14:00:00', '2022-10-26 11:00:00', '2023-01-10 10:00:00',
#     '2023-01-24 14:00:00', '2023-01-25 11:00:00', '2023-01-26 14:00:00', '2023-04-25 14:00:00',
#     '2023-04-26 11:00:00', '2023-06-06 11:00:00', '2023-07-25 14:00:00', '2023-07-26 14:00:00',
#     '2023-08-01 14:00:00', '2023-08-23 10:00:00', '2023-10-24 14:00:00', '2023-10-25 11:00:00',
#     '2023-12-05 16:00:00', '2023-12-11 11:00:00', '2024-01-23 14:00:00', '2024-01-24 11:00:00',
#     '2024-01-25 14:00:00', '2024-01-29 11:00:00', '2024-02-06 15:00:00', '2024-02-13 14:00:00',
#     '2024-02-20 11:00:00', '2024-02-27 10:00:00', '2024-03-05 13:00:00', '2024-03-12 09:00:00',
#     '2024-03-19 16:00:00', '2024-03-26 11:00:00', '2024-02-04 14:30:00'
# ]



# blocked_times = set(datetime.strptime(ts, "%Y-%m-%d %H:%M:%S") for ts in blocked_times_str)

# def is_blocked(ts):
#     return ts in blocked_times

# def random_email_time(before_time, min_hours_before=2, max_hours_before=48):
#     for _ in range(100):
#         delta_hours = random.randint(min_hours_before, max_hours_before)
#         candidate = before_time - timedelta(hours=delta_hours)
#         candidate = candidate.replace(minute=random.choice([0,15,30,45]), second=0, microsecond=0)
#         if not is_blocked(candidate):
#             return candidate
#     raise ValueError("Cannot find valid email timestamp")

# # Generate client contacts
# client_contacts = {}
# for client in first_meetings.keys():
#     name = fake.name()
#     clean_name = name.lower().replace(" ", ".").replace("'", "")
#     clean_client = client.lower().replace(" ", "").replace(".", "")
#     email = f"{clean_name}@{clean_client}.com"
#     client_contacts[client] = (name, email)

# # Meeting data generation
# rows_meetings = []
# for client, first_time_str in first_meetings.items():
#     first_time = datetime.strptime(first_time_str, "%Y-%m-%d %H:%M:%S")
#     for i, meeting_type in enumerate(meeting_types):
#         meeting_time = first_time + timedelta(days=14 * (i + 1))
#         rows_meetings.append({
#             "meeting_timestamp": meeting_time,
#             "host": "Jane Moneypenny",
#             "host_email": "jane.moneypenny@bankwell.com",
#             "invitees": f"{client} Treasury Team",
#             "invitee_emails": f"treasury@{client.lower().replace(' ', '').replace('.', '')}.com",
#             "meeting_subject": f"{client} - {meeting_type}"
#         })

# df_meetings = pd.DataFrame(rows_meetings)

# # Varied email templates for Jane
# jane_email_templates = {
#     "Regulatory Update": [
#         "Dear {client_name},\n\nI wanted to remind you of our upcoming regulatory update meeting. Let me know if there are any specific regulations you'd like to discuss.\n\nBest regards,\nJane",
#         "Hi {client_name},\n\nLooking forward to catching up on regulatory changes during our next meeting. Feel free to send any questions in advance.\n\nCheers,\nJane",
#         "Hello {client_name},\n\nOur next meeting will cover important regulatory updates. Please share any topics you want me to prepare.\n\nRegards,\nJane",
#         "Dear {client_name},\n\nPlease find attached a briefing note for our upcoming regulatory discussion. I’d appreciate any feedback on focus areas you care about.\n\nBest,\nJane",
#         "There are a few SEC-related updates I think will be relevant to our next conversation. Let me know if you want deeper analysis, {client_name}.\n\nCheers,\nJane",
#         "Good morning {client_name},\n\nI’ll be covering some cross-border compliance updates. Do you need a briefing on any non-U.S. jurisdictions?\n\nWarm regards,\nJane"
#     ],
#     "Risk Management": [
#         "Dear {client_name},\n\nAhead of our risk management session, please think about any concerns or risks you'd like to address.\n\nBest,\nJane",
#         "Hi {client_name},\n\nOur upcoming meeting focuses on risk management. Let me know if there are specific risks you want to explore.\n\nThanks,\nJane",
#         "Hello {client_name},\n\nJust touching base before our risk management discussion. Please share any risk areas you'd like to highlight.\n\nWarmly,\nJane",
#         "Looking forward to our discussion on current risk exposures, {client_name}. Please let me know if there are any specific areas you'd like me to prepare for.\n\nBest,\nJane",
#         "Hi {client_name},\n\nLet’s ensure our risk review is comprehensive—send over any recent developments or exposures we should cover.\n\nThanks,\nJane",
#         "We’ll be reviewing strategic risk metrics in our next meeting. Should I include simulations or historical stress events, {client_name}?\n\nBest,\nJane"
#     ],
#     "Annual Goal Alignment": [
#         "Dear {client_name},\n\nAs we prepare for our annual planning meeting, I’d love your input on strategic priorities.\n\nBest regards,\nJane",
#         "Hi {client_name},\n\nLooking forward to aligning on your 12-month goals. Let me know if you’d like me to prepare anything specific.\n\nThanks,\nJane",
#         "Hello {client_name},\n\nAhead of our annual goal alignment session, feel free to send along your draft targets or any focus areas.\n\nRegards,\nJane",
#         "Hi {client_name},\n\nLet’s come prepared with both tactical and long-term objectives—we’ll chart milestones together.\n\nCheers,\nJane",
#         "Good day {client_name},\n\nThinking through how we can measure progress against your top 3 goals—any insights before our call?\n\nSincerely,\nJane"
#     ],
#     "Tax Optimization": [
#         "Dear {client_name},\n\nI’ll walk you through some of the latest opportunities for tax deferral and loss harvesting. Let me know if there’s anything urgent.\n\nRegards,\nJane",
#         "Hi {client_name},\n\nLooking forward to exploring how we can reduce your tax burden this quarter. Please bring any recent tax docs if applicable.\n\nThanks,\nJane",
#         "Hello {client_name},\n\nLet’s dive into structured solutions and timing strategies in our upcoming session.\n\nBest,\nJane",
#         "There’s been an IRS guidance change that may open a new planning path, {client_name}. I’ll explain during our meeting.\n\nCheers,\nJane",
#         "If there’s any capital gains activity this year, let’s factor that into our discussion, {client_name}.\n\nWarm regards,\nJane"
#     ],
#     "ESG Investing": [
#         "Hi {client_name},\n\nI’ve gathered a few ESG-aligned funds that might suit your mandates. I’ll walk you through the performance vs. benchmarks.\n\nBest,\nJane",
#         "Dear {client_name},\n\nLet’s review ESG screening criteria in our meeting. Any new sustainability goals from your team I should know about?\n\nThanks,\nJane",
#         "Hello {client_name},\n\nI’ve been exploring green bond options—happy to show you what’s available this cycle.\n\nRegards,\nJane",
#         "Hi {client_name},\n\nWe’ll touch on ESG ratings and exclusions as part of our portfolio review.\n\nWarmly,\nJane",
#         "Good day {client_name},\n\nI’ve updated our ESG scoring dashboard and would love your feedback.\n\nSincerely,\nJane"
#     ],
#     "Macro Update": [
#         "Hi {client_name},\n\nI’ve pulled together key macroeconomic data and forecasts. Let’s walk through what may affect your portfolio.\n\nCheers,\nJane",
#         "Dear {client_name},\n\nAhead of our macro update, I’m including charts on inflation, rates, and global growth. Let me know if you'd like anything added.\n\nThanks,\nJane",
#         "Hello {client_name},\n\nPlenty to discuss this cycle—Fed outlook, global demand, and capital flows. I’ll give you a synthesis.\n\nWarm regards,\nJane",
#         "Expect updates on policy shifts and market trends in our next discussion, {client_name}.\n\nBest,\nJane",
#         "I’ll frame the macro view from a risk and opportunity lens. Do you want sector-level analysis too, {client_name}?\n\nSincerely,\nJane"
#     ],
#     "New Funds Offerings": [
#         "Hi {client_name},\n\nWe’ve launched two new funds—one growth-oriented and one fixed income. I’ll share details during the call.\n\nBest,\nJane",
#         "You might be interested in a new fund focused on sustainable yield, {client_name}. Let me know if you’d like materials in advance.\n\nThanks,\nJane",
#         "There’s a new thematic fund I’d like to flag—it might align with your current allocation, {client_name}.\n\nWarm regards,\nJane",
#         "Hi {client_name},\n\nAhead of the meeting, take a glance at the fund summary I attached. Excited to get your thoughts.\n\nCheers,\nJane",
#         "We’ll cover the new offerings next week. Bring any questions or mandates you're exploring, {client_name}.\n\nKindly,\nJane"
#     ],
#     "Benchmark Change": [
#         "Hi {client_name},\n\nThere’s been a proposed change to your benchmark index. Let’s review the rationale and impact during our call.\n\nBest,\nJane",
#         "I’ll show the side-by-side comparison of old vs. new benchmarks so we can validate fit, {client_name}.\n\nThanks,\nJane",
#         "Benchmark re-alignment can affect your performance metrics—I’ll walk you through it, {client_name}.\n\nWarm regards,\nJane",
#         "We’re proposing a shift to a blended index. I’ll share supporting analysis in our meeting, {client_name}.\n\nCheers,\nJane",
#         "We’ll cover historical tracking error and risk-return fit with the new benchmark, {client_name}.\n\nSincerely,\nJane"
#     ]
# }


# # Varied client questions templates keyed by meeting type
# client_email_templates = {
#     "Regulatory Update": [
#         "Hi Jane,\n\nThanks for setting this up. Can we include any upcoming FINRA changes in the discussion?\n\nBest,\n{client_name}",
#         "Dear Jane,\n\nLooking forward to the regulatory session. Could you touch on any state-level compliance updates too?\n\nRegards,\n{client_name}",
#         "Hello Jane,\n\nWould it be possible to review the new filing timelines as part of our meeting?\n\nThanks,\n{client_name}",
#         "Jane,\n\nCould we also discuss the latest anti-money laundering requirements?\n\nAppreciate it,\n{client_name}",
#         "Hi Jane,\n\nQuick question before our meeting—any recent shifts in cross-border disclosure rules we should note?\n\nBest,\n{client_name}"
#     ],
#     "Risk Management": [
#         "Hi Jane,\n\nWould love to explore potential FX exposure in our upcoming session.\n\nThanks,\n{client_name}",
#         "Dear Jane,\n\nCould you prepare something on our counterparty risk position as well?\n\nRegards,\n{client_name}",
#         "Hello Jane,\n\nJust checking if we’ll cover risk limits and tolerance thresholds during the session.\n\nThanks,\n{client_name}",
#         "Jane,\n\nCan you walk me through our current interest rate hedges again? Want to confirm alignment.\n\nBest,\n{client_name}",
#         "Hi Jane,\n\nAny updates on market risk scenarios that we should factor into our dashboard?\n\nCheers,\n{client_name}"
#     ],
#     "Annual Goal Alignment": [
#         "Hi Jane,\n\nFor our planning session, I’d like to explore quarterly targets aligned with our liquidity goals.\n\nBest,\n{client_name}",
#         "Dear Jane,\n\nCan we discuss metrics we should track against our new debt structure?\n\nThanks,\n{client_name}",
#         "Hello Jane,\n\nWould you be able to show projections tied to last year’s goal outcomes?\n\nWarmly,\n{client_name}",
#         "Jane,\n\nCould we bring in someone from your team who focuses on long-term capital planning?\n\nThanks,\n{client_name}",
#         "Hi Jane,\n\nI’d love to hear your view on aligning our treasury KPIs with broader finance objectives.\n\nRegards,\n{client_name}"
#     ],
#     "Tax Optimization": [
#         "Hi Jane,\n\nWould appreciate a review of potential deferral options for our year-end bonuses.\n\nThanks,\n{client_name}",
#         "Dear Jane,\n\nCould we factor in the new state tax rules when optimizing our setup?\n\nWarm regards,\n{client_name}",
#         "Jane,\n\nDo you have any modeling on short-term vs. long-term gains treatments?\n\nThanks,\n{client_name}",
#         "Hello Jane,\n\nOne quick request—let’s include charitable deductions in the analysis.\n\nBest,\n{client_name}",
#         "Hi Jane,\n\nCan you include tax drag impacts under current allocation?\n\nThanks,\n{client_name}"
#     ],
#     "ESG Investing": [
#         "Hi Jane,\n\nWill we look at ESG fund flows or just performance vs. traditional indices?\n\nThanks,\n{client_name}",
#         "Jane,\n\nCould you break down ESG screening criteria again, especially for energy holdings?\n\nRegards,\n{client_name}",
#         "Dear Jane,\n\nAny updates on impact-focused funds you’d recommend for our review?\n\nBest,\n{client_name}",
#         "Hello Jane,\n\nCan we also compare carbon scores across the proposed portfolios?\n\nCheers,\n{client_name}",
#         "Hi Jane,\n\nDo you have a view on integrating ESG in short-term instruments?\n\nThanks,\n{client_name}"
#     ],
#     "Macro Update": [
#         "Jane,\n\nCan we allocate a bit more time to global inflation forecasts?\n\nBest,\n{client_name}",
#         "Hi Jane,\n\nWould appreciate your take on central bank policy divergence.\n\nThanks,\n{client_name}",
#         "Dear Jane,\n\nCould you include a chart on commodity trends this cycle?\n\nRegards,\n{client_name}",
#         "Hello Jane,\n\nJust flagging—geopolitical risks may merit some attention this meeting.\n\nThanks,\n{client_name}",
#         "Hi Jane,\n\nAny chance we can look at cross-asset volatility in your macro outlook?\n\nCheers,\n{client_name}"
#     ],
#     "New Funds Offerings": [
#         "Hi Jane,\n\nCould you send over fund sheets in advance for review?\n\nThanks,\n{client_name}",
#         "Jane,\n\nDo any of the new funds support daily liquidity? That’s key for us.\n\nBest,\n{client_name}",
#         "Dear Jane,\n\nWould love to hear if there’s a low-volatility strategy among the new products.\n\nRegards,\n{client_name}",
#         "Hello Jane,\n\nI’m curious if any of the offerings lean into private credit or alternatives.\n\nThanks,\n{client_name}",
#         "Hi Jane,\n\nAre there ESG-screened versions of these funds as well?\n\nWarm regards,\n{client_name}"
#     ],
#     "Benchmark Change": [
#         "Jane,\n\nLooking forward to this one—can we dig into how the new benchmark affects our Sharpe ratio?\n\nThanks,\n{client_name}",
#         "Hi Jane,\n\nAny visuals you could share to show tracking differences?\n\nBest,\n{client_name}",
#         "Dear Jane,\n\nCan you walk me through sector weight shifts with the new benchmark?\n\nThanks,\n{client_name}",
#         "Hello Jane,\n\nI’m curious about volatility vs. the previous benchmark. Can we include that?\n\nCheers,\n{client_name}",
#         "Jane,\n\nJust flagging I’ll need to report this internally, so if there’s a summary memo, that’d be great.\n\nBest,\n{client_name}"
#     ]
# }

# # For meeting types without templates, fallback to a generic one
# def get_random_template(templates_dict, key, client_name):
#     templates = templates_dict.get(key)
#     if not templates:
#         # Generic fallback
#         return f"Dear {client_name},\n\nLooking forward to our discussion on {key}. Let me know if you want to cover anything specific.\n\nBest,\nJane"
#     else:
#         template = random.choice(templates)
#         return template.format(client_name=client_name)

# def get_random_question(questions_dict, key):
#     questions = questions_dict.get(key)
#     if not questions:
#         return "I have some questions regarding the meeting agenda."
#     else:
#         return random.choice(questions)

# # Generate emails DataFrame rows with varied text
# email_rows = []
# idx = 0

# for _, meeting in df_meetings.iterrows():
#     meeting_time = meeting['meeting_timestamp']
#     client = meeting['invitees'].replace(' Treasury Team', '')
#     meeting_type = meeting['meeting_subject'].split(' - ')[1]

#     client_name, client_email = client_contacts[client]

#     # --- Jane's initial email ---
#     jane_email_time = random_email_time(meeting_time, 2, 48)
#     jane_subject = f"{client} - {meeting_type}"
#     jane_body = get_random_template(jane_email_templates, meeting_type, client_name)

#     email_rows.append({
#         "index": idx,
#         "email_timestamp": jane_email_time,
#         "from_name": "Jane Moneypenny",
#         "from_email": "jane.moneypenny@bankwell.com",
#         "to_name": client_name,
#         "to_email": client_email,
#         "email_subject": jane_subject,
#         "email_body": jane_body,
#         "meeting_timestamp": meeting_time,
#         "meeting_subject": meeting['meeting_subject']
#     })
#     idx += 1

#     # --- Client's reply ---
#     min_reply_time = jane_email_time + timedelta(hours=1)
#     max_reply_time = meeting_time - timedelta(hours=1)

#     for _ in range(100):
#         if min_reply_time >= max_reply_time:
#             client_email_time = min_reply_time
#             break
#         delta_hours = random.randint(1, max(1, int((max_reply_time - min_reply_time).total_seconds() // 3600)))
#         client_email_time = min_reply_time + timedelta(hours=delta_hours)
#         client_email_time = client_email_time.replace(minute=random.choice([0,15,30,45]), second=0, microsecond=0)
#         if not is_blocked(client_email_time) and client_email_time < meeting_time:
#             break
#     else:
#         client_email_time = min_reply_time

#     client_subject = f"Re: {client} - {meeting_type}"
#     client_question = get_random_question(client_email_templates, meeting_type)
#     client_body = (
#         f"Dear Jane,\n\n"
#         f"Thank you for your email. {client_question}\n\n"
#         f"Best regards,\n{client_name}"
#     )

#     email_rows.append({
#         "index": idx,
#         "email_timestamp": client_email_time,
#         "from_name": client_name,
#         "from_email": client_email,
#         "to_name": "Jane Moneypenny",
#         "to_email": "jane.moneypenny@bankwell.com",
#         "email_subject": client_subject,
#         "email_body": client_body,
#         "meeting_timestamp": meeting_time,
#         "meeting_subject": meeting['meeting_subject']
#     })
#     idx += 1

# df_emails = pd.DataFrame(email_rows)
# df_emails = df_emails.sort_values("email_timestamp").reset_index(drop=True)

# #print(df_emails.head(1))


In [ ]:
# # Extract the company name before the hyphen in 'meeting_subject'

# # Extract company from meeting_subject
# df1['company'] = df1['meeting_subject'].str.split(' - ').str[0]

# # Create company-invitee pair column
# df1['company_invitee_pair'] = list(zip(df1['company'], df1['invitees'],df1['invitee_emails']))

# # Drop duplicate rows based on the pair
# df_unique = df1.drop_duplicates(subset=['company_invitee_pair']).reset_index(drop=True)

# # Show the result
# print(df_unique[['company', 'invitees', 'company_invitee_pair','invitee_emails']])

***Insert into SQL Database***

#meeting_data




In [28]:
#Meetings data

from sqlalchemy import text

# Convert the DataFrame to a list of dictionaries (each row = one dict)
records = df_meetings_combined.to_dict(orient="records")

# SQL INSERT query with named parameters
insert_query = """
    INSERT INTO meeting_data (
        meeting_timestamp,
        host,
        host_email,
        invitees,
        invitee_emails,
        meeting_subject
    ) VALUES (
        :meeting_timestamp,
        :host,
        :host_email,
        :invitees,
        :invitee_emails,
        :meeting_subject
    )
"""

# Insert all rows using the engine
with engine.begin() as conn:  # this handles commit automatically
    conn.execute(text(insert_query), records)

In [33]:
pd.set_option('display.max_columns', None)        # show all columns
pd.set_option('display.width', None)              # don't wrap lines
pd.set_option('display.max_colwidth', None)       # show full contents in each column

df1_adj = pd.read_sql("SELECT * FROM meeting_data", con=engine)
df1_adj

,index,meeting_timestamp,host,host_email,invitees,invitee_emails,meeting_subject
0,0.0,2018-07-17 15:00:00,Jane Moneypenny,jane.moneypenny@bankwell.com,Cynthia Hobbs,cynthia.hobbs@abbvie.com,AbbVie - Introductory Call & Bankwell Services Overview
1,1.0,2018-07-24 14:00:00,Jane Moneypenny,jane.moneypenny@bankwell.com,Jennifer Phelps,jennifer.phelps@aerovironment.com,AeroVironment - Initial Discussion on Financial Needs
2,2.0,2018-08-07 10:00:00,Jane Moneypenny,jane.moneypenny@bankwell.com,Kyle Waters,kyle.waters@amedisys.com,Amedisys - Discussing Cash Management Needs
3,3.0,2018-08-15 11:00:00,Jane Moneypenny,jane.moneypenny@bankwell.com,Kyle Waters,kyle.waters@amedisys.com,Amedisys - Cash Concentration Services Demo
4,4.0,2018-08-29 14:00:00,Jane Moneypenny,jane.moneypenny@bankwell.com,Denise Moore,denise.moore@celestica.com,Celestica - Initial Discussion on Liquidity Management
5,5.0,2018-09-25 10:00:00,Jane Moneypenny,jane.moneypenny@bankwell.com,Adam Clay,adam.clay@compass.com,Compass - Discuss Expansion Financing Needs
6,6.0,2018-10-11 16:00:00,Jane Moneypenny,jane.moneypenny@bankwell.com,Amy Winters,amy.winters@gamestopcorp.com,GameStop Corp - Merchant Services Initial Review
7,7.0,2018-11-05 16:00:00,Jane Moneypenny,jane.moneypenny@bankwell.com,Roberto Martin,roberto.martin@guardanthealth.com,Guardant Health - Corporate Investment Policy Review
8,8.0,2018-11-14 11:00:00,Jane Moneypenny,jane.moneypenny@bankwell.com,Jessica Palmer,jessica.palmer@hasbro.com,Hasbro - Discuss FX Risk Management Strategies (Forwards vs. Options)
9,9.0,2018-12-12 13:00:00,Jane Moneypenny,jane.moneypenny@bankwell.com,Michelle Jenkins,michelle.jenkins@intuitivesurgical.com,Intuitive Surgical - Q4 Check-in & Cash Management Follow-up


***Insert into SQL Database***

#email_data

In [34]:
#Meetings data

from sqlalchemy import text

# Convert the DataFrame to a list of dictionaries (each row = one dict)
records = df_emails_new.to_dict(orient="records")

# SQL INSERT query with named parameters
insert_query = """
    INSERT INTO email_data (
        email_timestamp,
        to_names,
        to_emails,
        from_name,
        from_email,
        email_subject,
        email_body
    ) VALUES (
        :email_timestamp,
        :to_names,
        :to_emails,
        :from_name,
        :from_email,
        :email_subject,
        :email_body
    )
"""

# Insert all rows using the engine
with engine.begin() as conn:  # this handles commit automatically
    conn.execute(text(insert_query), records)

In [36]:
df2_adj = pd.read_sql("SELECT * FROM email_data", con=engine)
df2_adj

,index,email_timestamp,to_names,to_emails,from_name,from_email,email_subject,email_body
0,0.0,2018-07-15 09:30:00,Cynthia Hobbs,cynthia.hobbs@abbvie.com,Jane Moneypenny,jane.moneypenny@bankwell.com,Introduction - Bankwell Financial Services for AbbVie,"Hi Cynthia,\n\nMy name is Jane Moneypenny, and I'm a Financial Advisor with Bankwell Financial, specializing in corporate financial solutions for the Healthcare sector. I came across your profile and wanted to reach out to see if AbbVie might benefit from our expertise in cash management, investment strategies, and risk mitigation.\n\nWould you be open to a brief introductory call sometime next week?\n\nBest regards,\n\n--\nJane Moneypenny\nFinancial Advisor\nBankwell Financial\nPhone: (555) 123-4567\nEmail: jane.moneypenny1@outlook.com\n"
1,1.0,2018-07-16 11:05:00,Jane Moneypenny,jane.moneypenny@bankwell.com,Cynthia Hobbs,cynthia.hobbs@abbvie.com,Re: Introduction - Bankwell Financial Services for AbbVie,"Hi Jane,\n\nThank you for reaching out. AbbVie is always interested in optimizing its financial operations. I'd be happy to schedule a brief call. How does Tuesday at 2 PM CT sound?\n\nBest,\n\n--\nCynthia Hobbs\nDirector\nAbbVie\nEmail: cynthia.hobbs@abbvie.com\nPhone: (446)673-8121x90878\n"
2,2.0,2018-07-16 11:15:00,Cynthia Hobbs,cynthia.hobbs@abbvie.com,Jane Moneypenny,jane.moneypenny@bankwell.com,Re: Introduction - Bankwell Financial Services for AbbVie,"Hi Cynthia,\n\nTuesday at 2 PM CT works perfectly. I'll send a calendar invitation shortly. Looking forward to our conversation.\n\nBest regards,\n\n--\nJane Moneypenny\nFinancial Advisor\nBankwell Financial\nPhone: (555) 123-4567\nEmail: jane.moneypenny1@outlook.com\n"
3,3.0,2018-08-01 14:00:00,Jennifer Phelps,jennifer.phelps@aerovironment.com,Jane Moneypenny,jane.moneypenny@bankwell.com,Follow Up: AeroVironment Financial Needs,"Hi Jennifer,\n\nIt was a pleasure speaking with you last week regarding AeroVironment's financial strategy, particularly around FX risk management for your international components sourcing. \n\nAs discussed, I've attached a preliminary overview of Bankwell's currency hedging solutions that could be relevant. Let me know if you have any initial questions or would like to dive deeper into specific options.\n\nBest regards,\n\n--\nJane Moneypenny\nFinancial Advisor\nBankwell Financial\nPhone: (555) 123-4567\nEmail: jane.moneypenny1@outlook.com\n"
4,4.0,2018-08-02 10:15:00,Jane Moneypenny,jane.moneypenny@bankwell.com,Jennifer Phelps,jennifer.phelps@aerovironment.com,Re: Follow Up: AeroVironment Financial Needs,"Hi Jane,\n\nThanks for sending this over. The overview is helpful. We're particularly interested in forward contracts given our predictable payment cycles. Could you provide some indicative pricing based on recent EUR/USD rates for 3 and 6-month terms?\n\nThanks,\n\n--\nJennifer Phelps\nSenior Director\nAeroVironment\nEmail: jennifer.phelps@aerovironment.com\nPhone: 797-584-0061x89137\n"
5,5.0,2018-08-02 16:30:00,Jennifer Phelps,jennifer.phelps@aerovironment.com,Jane Moneypenny,jane.moneypenny@bankwell.com,Re: Follow Up: AeroVironment Financial Needs,"Hi Jennifer,\n\nCertainly. I've run indicative pricing based on today's market close for EUR/USD forwards. Please see the attached summary for 3 and 6-month terms. Keep in mind these are indicative and subject to market fluctuations until execution.\n\nHappy to discuss further.\n\nBest regards,\n\n--\nJane Moneypenny\nFinancial Advisor\nBankwell Financial\nPhone: (555) 123-4567\nEmail: jane.moneypenny1@outlook.com\n"
6,6.0,2018-08-10 11:00:00,Kyle Waters,kyle.waters@amedisys.com,Jane Moneypenny,jane.moneypenny@bankwell.com,Bankwell Financial - Cash Concentration Services,"Hi Kyle,\n\nFollowing up on our conversation about streamlining Amedisys' cash management across your various operating accounts, I wanted to provide more details on Bankwell's automated cash concentration services.\n\nThe attached document outlines the potential benefits, includ